### DataLoading 

In [1]:
from __future__ import annotations
from pathlib import Path
from typing import Dict, List, Literal, Union
import pandas as pd

# ---------------------------------------------------------------------------
# Module-level docs & type aliases
# ---------------------------------------------------------------------------

"""
Generic loaders & converters for regional-timeseries data.

Canonical representation
------------------------
    CanonicalData = dict[str, pd.DataFrame]
        key   – region name
        value – DataFrame indexed by datetime; columns = features

Disk layouts we support
-----------------------
    "multi" : MultiIndex columns (level-0 = region, level-1 = feature)
    "long"  : Normal DataFrame with a *region* column
    "wide"  : Wide DataFrame whose column names are "<region><sep><feature>"
"""

CanonicalData = Dict[str, pd.DataFrame]
Layout        = Literal["multi", "long", "wide"]

# ---------------------------------------------------------------------------
# ─── Converters: DISK ➜ CANONICAL ────────────────────────────────────────────
# ---------------------------------------------------------------------------

def _multi_to_dict(
    df: pd.DataFrame,
    *,
    cols: List[str] | None = None
) -> CanonicalData:
    """
    Convert a Multi-Index column DataFrame to the canonical dict form.

    Parameters
    ----------
    df   : DataFrame with two column levels (region, feature)
    cols : optional list of feature names to *keep* (others are dropped)

    Returns
    -------
    Dict[str, DataFrame]  mapping region → feature-matrix
    """
    out: CanonicalData = {}
    for region in df.columns.get_level_values(0).unique():
        sub = df.xs(region, axis=1, level=0)
        if cols is not None:
            sub = sub[[c for c in cols if c in sub.columns]]
        out[region] = sub.copy()
    return out


def _long_to_dict(
    df: pd.DataFrame,
    *,
    region_col: str = "region",
    cols: List[str] | None = None,
) -> CanonicalData:
    """
    Convert a ‘long’ (tidy) DataFrame to canonical dict form.

    A *long* frame must contain a `region_col` column; all other columns
    are interpreted as features.

    Parameters
    ----------
    df         : tidy DataFrame with duplicate timestamps per region
    region_col : column that identifies the region
    cols       : optional feature filter

    Returns
    -------
    Canonical dict keyed by region
    """
    if region_col not in df.columns:
        raise KeyError(f"Expected column '{region_col}' in long-layout dataframe.")

    features = [c for c in df.columns if c != region_col]
    if cols is not None:
        features = [c for c in features if c in cols]

    out: CanonicalData = {}
    for region, grp in df.groupby(region_col):
        frame = grp.drop(columns=region_col)
        out[region] = frame[features].copy()
    return out


def _wide_to_dict(
    df: pd.DataFrame,
    *,
    sep: str = "-",
    cols: List[str] | None = None,
) -> CanonicalData:
    """
    Convert a wide DataFrame with ‘region-feature’ column names into canon dict.

    Parameters
    ----------
    df   : DataFrame whose columns look like "tokyo-demand"
    sep  : separator between region and feature
    cols : optional feature filter

    Returns
    -------
    Canonical dict keyed by region
    """
    parts = df.columns.to_series().str.split(sep, n=1, expand=True)
    if parts.isna().any().any():
        raise ValueError(
            f"Column names do not all match <region>{sep}<feature> pattern."
        )

    out: CanonicalData = {}
    for region in parts[0].unique():
        mask      = parts[0] == region
        sub_cols  = df.columns[mask]
        features  = parts[1][mask]
        sub_frame = df[sub_cols].copy()
        sub_frame.columns = features
        if cols is not None:
            sub_frame = sub_frame[[c for c in cols if c in sub_frame.columns]]
        out[region] = sub_frame
    return out

# ---------------------------------------------------------------------------
# ─── Converters: CANONICAL ➜ DISK ────────────────────────────────────────────
# ---------------------------------------------------------------------------

def _dict_to_multi(data: CanonicalData) -> pd.DataFrame:
    """
    Combine a canonical dict into a Multi-Index column DataFrame.
    """
    frames = []
    for region, df in data.items():
        tmp = df.copy()
        tmp.columns = pd.MultiIndex.from_product([[region], tmp.columns])
        frames.append(tmp)
    return pd.concat(frames, axis=1).sort_index(axis=1)


def _dict_to_long(
    data: CanonicalData,
    *,
    region_col: str = "region"
) -> pd.DataFrame:
    """
    Combine a canonical dict into a long (tidy) DataFrame.
    """
    frames = []
    for region, df in data.items():
        tmp = df.copy()
        tmp[region_col] = region
        frames.append(tmp)
    combined = pd.concat(frames)
    order = [c for c in combined.columns if c != region_col] + [region_col]
    return combined[order]


def _dict_to_wide(
    data: CanonicalData,
    *,
    sep: str = "-"
) -> pd.DataFrame:
    """
    Combine a canonical dict into wide ‘region-feature’ columns.
    """
    frames = []
    for region, df in data.items():
        tmp = df.copy()
        tmp.columns = [f"{region}{sep}{c}" for c in tmp.columns]
        frames.append(tmp)
    return pd.concat(frames, axis=1)

# ---------------------------------------------------------------------------
# ─── Public helpers: one-file load / convert ────────────────────────────────
# ---------------------------------------------------------------------------

def load_parquet_as_canonical(
    path: str | Path,
    *,
    layout: Layout,
    region_col: str = "region",
    sep: str = "-",
    cols: List[str] | None = None,
) -> CanonicalData:
    """
    Read a parquet file of known *layout* and return canonical dict form.

    Parameters
    ----------
    path       : file path
    layout     : "multi", "long", or "wide" (must match the file)
    region_col : name of the region column for *long* layout
    sep        : region-feature separator for *wide* layout
    cols       : optional feature subset

    Returns
    -------
    CanonicalData
    """
    df = pd.read_parquet(path)
    if layout == "multi":
        return _multi_to_dict(df, cols=cols)
    if layout == "long":
        return _long_to_dict(df, region_col=region_col, cols=cols)
    if layout == "wide":
        return _wide_to_dict(df, sep=sep, cols=cols)
    raise ValueError(f"Unsupported layout '{layout}'.")


def canonical_to_layout(
    data: CanonicalData,
    layout: Layout,
    *,
    region_col: str = "region",
    sep: str = "-",
) -> pd.DataFrame:
    """
    Convert *canonical* dict back to the requested layout format.

    Parameters
    ----------
    data       : canonical dict
    layout     : target layout ("multi", "long", "wide")
    region_col : name of region column for long layout
    sep        : separator for wide layout

    Returns
    -------
    DataFrame in the specified layout
    """
    if layout == "multi":
        return _dict_to_multi(data)
    if layout == "long":
        return _dict_to_long(data, region_col=region_col)
    if layout == "wide":
        return _dict_to_wide(data, sep=sep)
    raise ValueError(f"Unsupported target layout '{layout}'.")

# ---------------------------------------------------------------------------
# ─── Helpers for multi-file ingestion ───────────────────────────────────────
# ---------------------------------------------------------------------------

def _collect_by_layout(
    paths: List[str] | None,
    layout: Layout,
    *,
    cols: List[str] | None,
    region_col: str,
    sep: str,
) -> Dict[str, List[pd.DataFrame]]:
    """
    Internal helper: load every file in `paths` (given its layout)
    and collate DataFrames by region.

    Returns
    -------
    Dict[region, list[DataFrame]]
    """
    bucket: Dict[str, List[pd.DataFrame]] = {}
    for p in paths or []:
        data = load_parquet_as_canonical(
            p, layout=layout, region_col=region_col, sep=sep, cols=cols
        )
        for region, df in data.items():
            bucket.setdefault(region, []).append(df)
    return bucket


def load_region_dataset(
    *,
    region: str,
    multi_paths: List[str] | None = None,
    long_paths: List[str] | None = None,
    wide_paths: List[str] | None = None,
    cols: List[str] | None = None,
    region_col: str = "region",
    sep: str = "-",
) -> pd.DataFrame:
    """
    Aggregate **one** region’s data from any number of parquet files.

    Parameters
    ----------
    region      : name of the region to extract
    multi_paths : list of files in *multi* layout
    long_paths  : list of files in *long* layout
    wide_paths  : list of files in *wide* layout
    cols        : optional feature subset
    region_col  : region column (long layout)
    sep         : separator (wide layout)

    Returns
    -------
    DataFrame with union of timestamps & features for that region
    (duplicate columns are de-duplicated, keeping last-read version).
    """
    frames: List[pd.DataFrame] = []
    for layout, paths in (
        ("multi", multi_paths),
        ("long", long_paths),
        ("wide", wide_paths),
    ):
        for p in paths or []:
            data = load_parquet_as_canonical(
                p, layout=layout, region_col=region_col, sep=sep, cols=cols
            )
            if region in data:
                frames.append(data[region])

    if not frames:
        raise KeyError(f"Region '{region}' not found in supplied paths.")

    combined = pd.concat(frames, axis=1)
    combined = combined.loc[:, ~combined.columns.duplicated()]  # keep last duplicate
    combined.sort_index(inplace=True)
    return combined


def load_all_regions_dataset(
    *,
    multi_paths: List[str] | None = None,
    long_paths: List[str] | None = None,
    wide_paths: List[str] | None = None,
    regions: List[str] | None = None,
    cols: List[str] | None = None,
    region_col: str = "region",
    sep: str = "-",
) -> CanonicalData:
    """
    Load **every** region (or a specified subset) from the supplied file lists.

    Parameters
    ----------
    multi_paths : parquet files in *multi* layout
    long_paths  : parquet files in *long* layout
    wide_paths  : parquet files in *wide* layout
    regions     : optional subset of region names to return
    cols        : optional feature subset
    region_col  : region column (long layout)
    sep         : separator (wide layout)

    Returns
    -------
    CanonicalData containing one DataFrame per region.
    """
    # Read & collate per-layout
    bucket: Dict[str, List[pd.DataFrame]] = {}

    for paths, layout in (
        (multi_paths, "multi"),
        (long_paths, "long"),
        (wide_paths, "wide"),
    ):
        part = _collect_by_layout(
            paths,
            layout,
            cols=cols,
            region_col=region_col,
            sep=sep,
        )
        for region, frames in part.items():
            bucket.setdefault(region, []).extend(frames)

    # Filter to requested subset (if any)
    if regions is not None:
        bucket = {r: bucket[r] for r in regions if r in bucket}

    # Merge frames per region
    out: CanonicalData = {}
    for region, frames in bucket.items():
        df = pd.concat(frames, axis=1)
        df = df.loc[:, ~df.columns.duplicated()]
        df.sort_index(inplace=True)
        out[region] = df

    # Handle missing regions gracefully
    if regions is not None:
        missing = set(regions) - set(out)
        if missing:
            raise KeyError(
                f"The following requested regions were not found in any file: "
                f"{', '.join(sorted(missing))}"
            )

    return out
#88888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888

from __future__ import annotations
from pathlib import Path
from typing import Dict, List, Literal
import pandas as pd

# -------------  Canonical type & layout tags  --------------------------------
CanonicalData = Dict[str, pd.DataFrame]
Layout        = Literal["multi", "long", "wide"]

# ----------------------------------------------------------------------------- 
# ═════════════════════════════ I/O CONVERSION ════════════════════════════════
# (unchanged from earlier — trimmed for brevity; keep the same helper fns:
#     _multi_to_dict, _long_to_dict, _wide_to_dict,
#     _dict_to_multi,  _dict_to_long,  _dict_to_wide,
#     load_parquet_as_canonical, canonical_to_layout)
# ----------------------------------------------------------------------------- 

# ...  <keep earlier converter code here exactly as before> ...


# ----------------------------------------------------------------------------- 
# ══════════════════════════ PRE-MERGE FEATURE PASS ═══════════════════════════
# ----------------------------------------------------------------------------- 

from jp_da_imb.utils.time import construct_time_features   # ← your existing util


def preprocess_region_df(
    df: pd.DataFrame,
    *,
    start_date: str | pd.Timestamp | None = None,
    end_date:   str | pd.Timestamp | None = None,
    freq: str | None = "30T",
    na_removal: bool = True,
    add_time_feats: bool = True,
) -> pd.DataFrame:
    """
    Apply *per-file* hygiene & feature engineering **before** merging with other
    DataFrames for the same region.

    Steps (all optional, controlled by kwargs)
    -----------------------------------------
    1. Clip to [start_date, end_date] (if provided)
    2. Resample to `freq` (mean aggregation)       — skip if `freq=None`
    3. Drop rows with any NA                       — only if `na_removal=True`
    4. Add calendar/time features (and cast them
       to `category`) using your `construct_time_features`
       — only if `add_time_feats=True`
    """
    out = df.copy()

    # 1. Clip date range -------------------------------------------------------
    if start_date is not None:
        out = out.loc[out.index >= pd.to_datetime(start_date)]
    if end_date is not None:
        out = out.loc[out.index <= pd.to_datetime(end_date)]

    # 2. Resample --------------------------------------------------------------
    if freq is not None:
        out = out.resample(freq).mean()

    # 3. Remove NAs ------------------------------------------------------------
    if na_removal:
        out = out.dropna()

    # 4. Calendar / categorical time features ---------------------------------
    if add_time_feats:
        construct_time_features(out)     # mutates in place, adds columns
        time_cols = [
            "weekday", "hour", "month", "quarter",
            "koma", "koma_week", "is_holiday",
            "is_peak", "is_weekend",
        ]
        for c in time_cols:
            if c in out.columns and not pd.api.types.is_categorical_dtype(out[c]):
                out[c] = pd.Categorical(out[c])

    return out


# ----------------------------------------------------------------------------- 
# ═══════════════  MULTI-FILE INGEST, NOW WITH PRE-PASS  ══════════════════════
# ----------------------------------------------------------------------------- 

def _collect_by_layout(
    paths: List[str] | None,
    layout: Layout,
    *,
    cols: List[str] | None,
    region_col: str,
    sep: str,
    preprocess_kwargs: dict,
) -> Dict[str, List[pd.DataFrame]]:
    """
    Load every parquet file in `paths` -> canonical dict,
    run `preprocess_region_df` on each region DataFrame,
    and bucket them by region.
    """
    bucket: Dict[str, List[pd.DataFrame]] = {}
    for p in paths or []:
        data = load_parquet_as_canonical(
            p, layout=layout, region_col=region_col, sep=sep, cols=cols
        )
        for region, df in data.items():
            df_proc = preprocess_region_df(df, **preprocess_kwargs)
            bucket.setdefault(region, []).append(df_proc)
    return bucket


def load_region_dataset(
    *,
    region: str,
    multi_paths: List[str] | None = None,
    long_paths:  List[str] | None = None,
    wide_paths:  List[str] | None = None,
    cols:       List[str] | None = None,
    region_col: str = "region",
    sep: str = "-",
    # --- pre-processing options ----------------------------------------------
    start_date: str | pd.Timestamp | None = None,
    end_date:   str | pd.Timestamp | None = None,
    freq: str | None = "30T",
    na_removal: bool = True,
    add_time_feats: bool = True,
) -> pd.DataFrame:
    """
    Build a **single-region** DataFrame by reading any combination of
    *multi*, *long*, and *wide* parquet files, applying the pre-processing
    steps to each file **before** they are merged together.

    Other parameters are identical to the earlier version, plus the
    pre-processing kwargs shown above.
    """
    preprocess_kwargs = dict(
        start_date=start_date,
        end_date=end_date,
        freq=freq,
        na_removal=na_removal,
        add_time_feats=add_time_feats,
    )

    frames: List[pd.DataFrame] = []
    for layout, paths in (
        ("multi", multi_paths),
        ("long",  long_paths),
        ("wide",  wide_paths),
    ):
        for p in paths or []:
            data = load_parquet_as_canonical(
                p, layout=layout, region_col=region_col, sep=sep, cols=cols
            )
            if region in data:
                frames.append(preprocess_region_df(data[region], **preprocess_kwargs))

    if not frames:
        raise KeyError(f"Region '{region}' not found in supplied paths.")

    combined = pd.concat(frames, axis=1)
    combined = combined.loc[:, ~combined.columns.duplicated()]  # keep last dup
    combined.sort_index(inplace=True)
    return combined


def load_all_regions_dataset(
    *,
    multi_paths: List[str] | None = None,
    long_paths:  List[str] | None = None,
    wide_paths:  List[str] | None = None,
    regions:     List[str] | None = None,
    cols:       List[str] | None = None,
    region_col: str = "region",
    sep: str = "-",
    # --- pre-processing options ----------------------------------------------
    start_date: str | pd.Timestamp | None = None,
    end_date:   str | pd.Timestamp | None = None,
    freq: str | None = "30T",
    na_removal: bool = True,
    add_time_feats: bool = True,
) -> CanonicalData:
    """
    Load **all** regions (or a given subset) from the provided file lists,
    applying the pre-merge feature-creation steps to every individual file.
    """
    preprocess_kwargs = dict(
        start_date=start_date,
        end_date=end_date,
        freq=freq,
        na_removal=na_removal,
        add_time_feats=add_time_feats,
    )

    bucket: Dict[str, List[pd.DataFrame]] = {}

    for paths, layout in (
        (multi_paths, "multi"),
        (long_paths,  "long"),
        (wide_paths,  "wide"),
    ):
        part = _collect_by_layout(
            paths,
            layout,
            cols=cols,
            region_col=region_col,
            sep=sep,
            preprocess_kwargs=preprocess_kwargs,
        )
        for region, frames in part.items():
            bucket.setdefault(region, []).extend(frames)

    # subset filter
    if regions is not None:
        bucket = {r: bucket[r] for r in regions if r in bucket}

    # merge frames per region
    out: CanonicalData = {}
    for region, frames in bucket.items():
        df = pd.concat(frames, axis=1)
        df = df.loc[:, ~df.columns.duplicated()]
        df.sort_index(inplace=True)
        out[region] = df

    # raise on missing explicit requests
    if regions is not None:
        missing = set(regions) - set(out)
        if missing:
            raise KeyError(
                f"The following requested regions were not found in any file: "
                f\"{', '.join(sorted(missing))}\"
            )

    return out
